In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
plt.rcParams['figure.facecolor'] = "white"

from reservoir import reservoir as res
from data import data
from error import error

from reservoir.reservoir import modify_node

In [35]:
DEL_T = 0.01
STEPS=30000
WASHOUT=20000

# number of reservoir nodes
N = 100
# dimension of input signal
d = 3

In [36]:
hyperparams = {
    'GAMMA': 7.7,
    'SIGMA': 0.81,
    'RHO_IN': 0.37,
    'K': 3,
    'RHO_R': 0.41,
}

In [37]:
GAMMA_RANGE = np.linspace(7, 11, 10)
SIGMA_RANGE = np.linspace(0.1, 1.0, 10)
RHO_IN_RANGE = np.linspace(0.3, 1.5, 10)
K_RANGE = range(1, 6)
RHO_R_RANGE = np.linspace(0.3, 1.5, 10)

In [38]:
from typing import Type, Optional, Union, Tuple

def evaluate_forecast(reservoir_size: int, alpha: np.double, hyperparams: dict, 
                      training_data: np.ndarray, test_data: np.ndarray, seed: int, 
                      adjust_for_symmetry: bool = True) -> Tuple[np.ndarray]:
    """
    Computes the forecast error of a reservoir network generated using the given 
    training and test data. 

    Args:
        reservoir_size (int): the number of nodes in the reservoir network.
        alpha (np.double): the Tikhonov regularisation constant.
        hyperparams (dict): the hyperparameters used to initiate the reservoir.
        training_data (np.ndarray): the training data, of shape n1 x d.  
        test_data (np.ndarray): the test data, of shape n2 x d.
        seed (int): seed for reproduceability. 
        adjust_for_symmetry (bool): whether or not the network needs to be 
            transformed to account for the symmetry in the underlying input 
            signal series. 

            Defaults to True. 
    
    Returns:
        (Tuple): a tuple containing:
            (np.ndarray) the forecast error of the predictions. 
            (np.ndarray) the forecasted values.
            (np.ndarray) the forecast reservoir network.
            (np.ndarray) the training reservoir network.
            (np.ndarray) W_out, the output linear transformation.
            (np.ndarray) W_r, the internal connection network.
            (np.ndarray) W_in, the input signal to node connection network.
    """
    # compute dimensions
    N = reservoir_size
    n1 = training_data.shape[0]
    n2 = test_data.shape[0]
    d = training_data.shape[1]

    # initiate reservoir internals
    W_r = res.generate_W_r(hyperparams, (N, N), seed=seed)
    W_in = res.generate_W_in(hyperparams, (N, d), seed=seed)

    # train W_out
    training_res = res.generate_training_reservoir(
        data=training_data,
        hyperparams=hyperparams,
        W_r=W_r,
        W_in=W_in,
        delta_t=DEL_T,
        adjust_for_symmetry=adjust_for_symmetry
    )
    W_out = res.generate_W_out(data=training_data, res=training_res, alpha=alpha)

    # make forecast
    forecast_res = res.generate_forecast_reservoir(
        r_0=np.dot(W_in, test_data[0]),
        data=test_data,
        hyperparams=hyperparams,
        W_r=W_r,
        W_in=W_in,
        W_out=W_out,
        delta_t=DEL_T,
        adjust_for_symmetry=adjust_for_symmetry
    )

    preds = res.readout_network(forecast_res, W_out)

    return (error.RMSE(test_data, preds), preds, forecast_res, training_res, 
            W_out, W_r, W_in)

In [39]:
train_data = data.generate_lorenz_63(del_t=DEL_T, steps=STEPS, washout=WASHOUT)
test_data = data.generate_lorenz_63(
    initial_state=np.random.choice(np.linspace(-10, 10), 3),
    del_t=DEL_T, steps=STEPS, washout=WASHOUT
)
ALPHA = 0.001
trial = evaluate_forecast(
    reservoir_size=N,
    alpha=ALPHA,
    hyperparams=hyperparams,
    training_data=train_data,
    test_data=test_data,
    seed=42,
    adjust_for_symmetry=True
)

In [40]:
# search_space_size = len(GAMMA_RANGE) * len(SIGMA_RANGE) * len(RHO_IN_RANGE) * len(K_RANGE) * len(RHO_R_RANGE)
search_space_size = len(GAMMA_RANGE) * len(RHO_IN_RANGE) * len(RHO_R_RANGE)
print("Search space: {}".format(search_space_size))

Search space: 1000


In [42]:
trials_hyperparams = []
trials = []
count = 1

for gamma in GAMMA_RANGE:
    for rho_in in RHO_IN_RANGE:
        for rho_r in RHO_R_RANGE:
            hyperparams = {
                "GAMMA": gamma,
                "SIGMA": 0.5,
                "RHO_IN": rho_in,
                "K": 3,
                "RHO_R": rho_r
            }

            trials_hyperparams.append(hyperparams)
            
            trials.append(evaluate_forecast(
                reservoir_size=N,
                alpha=ALPHA,
                hyperparams=hyperparams,
                training_data=train_data,
                test_data=test_data,
                seed=42,
                adjust_for_symmetry=True
            ))

            print("Done: {} / {}".format(count, search_space_size))
            count += 1

Done: 1 / 1000
Done: 2 / 1000
Done: 3 / 1000
Done: 4 / 1000
Done: 5 / 1000
Done: 6 / 1000
Done: 7 / 1000
Done: 8 / 1000
Done: 9 / 1000
Done: 10 / 1000
Done: 11 / 1000
Done: 12 / 1000
Done: 13 / 1000
Done: 14 / 1000
Done: 15 / 1000
Done: 16 / 1000
Done: 17 / 1000
Done: 18 / 1000
Done: 19 / 1000
Done: 20 / 1000
Done: 21 / 1000
Done: 22 / 1000
Done: 23 / 1000
Done: 24 / 1000
Done: 25 / 1000
Done: 26 / 1000
Done: 27 / 1000
Done: 28 / 1000
Done: 29 / 1000
Done: 30 / 1000
Done: 31 / 1000
Done: 32 / 1000
Done: 33 / 1000
Done: 34 / 1000
Done: 35 / 1000
Done: 36 / 1000
Done: 37 / 1000
Done: 38 / 1000
Done: 39 / 1000
Done: 40 / 1000
Done: 41 / 1000
Done: 42 / 1000
Done: 43 / 1000
Done: 44 / 1000
Done: 45 / 1000
Done: 46 / 1000
Done: 47 / 1000
Done: 48 / 1000
Done: 49 / 1000
Done: 50 / 1000
Done: 51 / 1000
Done: 52 / 1000
Done: 53 / 1000
Done: 54 / 1000
Done: 55 / 1000
Done: 56 / 1000
Done: 57 / 1000
Done: 58 / 1000
Done: 59 / 1000
Done: 60 / 1000
Done: 61 / 1000
Done: 62 / 1000
Done: 63 / 1000
D

KeyboardInterrupt: 